In [1]:
using Pkg; for p in ("Embeddings","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra,FileIO
using Knet: Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %14.1 %>                            ]  28.2 % [=================>                       ]  42.4 %]  56.4 % [=============================>           ]  70.5 %81.0 %> ]  95.1 % Resolving package versions...
 Installed Ratios ────────────── v0.3.1
 Installed Widgets ───────────── v0.5.1
 Installed CUDAapi ───────────── v0.6.2
 Installed AutoHashEquals ────── v0.2.0
 Installed TranscodingStreams ── v0.9.2
 Installed Embeddings ────────── v0.3.0
 Installed DocStringExtensions ─ v0.7.0
 Installed WebSockets ────────── v1.5.0
 Installed SoftGlobalScope ───── v1.0.10
 Installed LibCURL ───────────── v0.5.0
 Installed HTTP ──────────────── v0.8.0
 Installed Tables ────────────── v0.1.18
 Installed DataDeps ──────────── v0.6.2
 Installed Crayons ───────────── v3.0.0
  Updating `~/.julia/environments/v1.1/Projec

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1186
┌ Info: Precompiling Embeddings [c5bfea45-b7f1-5224-a596-15500f5db411]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1184


In [2]:
mutable struct node 
    word
    kidsword
    kidsindex
    parent
    finished
    is_word
    selfindex
    parentindex
    label
    ind
node(word) = word == nothing ? new(nothing,nothing,nothing,nothing,nothing,0,nothing,nothing,nothing,nothing) : new(word,[],[],[],0,1,0,0,"",-1)
    
end

In [3]:
function get_split(size=5952)
dict=Dict()
    for i in range(1,length=size)
        if i < 5452
            dict[i] = 1
        else
            dict[i] =2
        end 
    end 
    return dict
end

get_split (generic function with 2 methods)

In [4]:
function get_labels(fn)
    f=open(fn,"r")
    dict=Dict()
    for (index, i) in enumerate(readlines(f))
        dict[index] = parse(Int,i) 
    end
    return dict
end

get_labels (generic function with 1 method)

In [5]:
function clean_str(string, TREC=false)
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    
    string = replace(string,r"[^A-Za-z0-9(),!?\'\`]" =>s" ")
    string = replace(string,r"\'s" =>s" 's") 
    string = replace(string,r"\'ve" =>s" 've") 
    string = replace(string,r"n\'t" =>s" n't") 
    string = replace(string,r"\'re" =>s" 're") 
    string = replace(string,r"\'d" =>s" 'd") 
    string = replace(string,r"\'ll" =>s" 'll") 
    string = replace(string,r"," =>s" , ") 
    string = replace(string,r"!" =>s" ! ") 
    string = replace(string,r"\(" =>s" \\( ") 
    string = replace(string,r"\)" =>s" \\) ") 
    string = replace(string,r"\?" =>s" \\? ") 
    string = replace(string,r"\s{2,}" =>s" ")    
        
    return (TREC ?  strip(string) : lowercase(strip(string)))
end

clean_str (generic function with 2 methods)

In [6]:
function build_data_cv(file, split_dict, label_dict, clean_string=false)
    """
    Loads data and split data
    """
    revs = []
    f = open(file,"r")
    vocab = DefaultDict(0)#https://juliacollections.github.io/DataStructures.jl/latest/default_dict.html
    
    for (index, line) in enumerate(readlines(f))     
        rev = []
        push!(rev,strip(line))
        if clean_string
            orig_rev = clean_str(join(rev," "))
        else
            orig_rev = join(rev," ")
        end
        words = Set(split(orig_rev))
        for word in words
            vocab[word] += 1
        end
        datum  = Dict("y"=>label_dict[index], 
                    "text"=> orig_rev,                             
                    "num_words"=> length(split(orig_rev)),
                    "split"=> split_dict[index])
        push!(revs,datum)
    end

    return revs, vocab
end

build_data_cv (generic function with 2 methods)

In [7]:
function sibling2(sents, opt)
    sent_list = []
    
    for (key,currnet_node) in sents
        #@show key
        #@show currnet_node
        if key == 0;continue;end
        #currnet_node = sents[key]
        word_list = []
        push!(word_list,currnet_node.word)
        
        parent_index = currnet_node.parentindex
        parent = sents[parent_index]
        push!(word_list,parent.word)
        sib_list = parent.kidsindex
        if key < parent_index
            sib_candidate = [i for i in sib_list if i < key]
            if sib_candidate == [];push!(word_list,"*START*")
            else;push!(word_list,sents[pop!(sib_candidate)].word);end 
            if sib_candidate == [];push!(word_list,"*START*")
            else;push!(word_list,sents[pop!(sib_candidate)].word);end
        else
            sib_candidate = [i for i in sib_list if i > key]
            if sib_candidate == [];push!(word_list,"*STOP*")
            else;push!(word_list,sents[pop!(sib_candidate)].word);end
            if sib_candidate == [];push!(word_list,"*STOP*")
            else;push!(word_list,sents[pop!(sib_candidate)].word); end
       end
        grad_parent_ind = parent.parentindex
        grad_word = sents[grad_parent_ind].word
        push!(word_list,grad_word)
        push!(sent_list,word_list)
    end
    return sent_list
end
                                                    

sibling2 (generic function with 1 method)

In [8]:
function set_sibling2(tree,labels_dict,max_len)

    sent_num = length(tree)
    doc_list =[]
    for (ind,sents) in enumerate(tree)
        #sents
        sib_6 = sibling2(sents,6)
        sent_list = sib_6
        dummy_len = length(sent_list[1])
        dummy = repeat(["*ZERO*"],dummy_len)
        while length(sent_list) < max_len
            push!(sent_list,dummy)
        end
        currnet_label = labels_dict[ind]
        class_dummy = [currnet_label]*dummy_len
        push!(sent_list,class_dummy)        
        push!(doc_list,sent_list)
    end
    return doc_list    
end

set_sibling2 (generic function with 1 method)

In [9]:
function set_conv_sent(tree,labels_dict,max_len)
    conv_length = 5
    @show sent_num = length(tree)
    ##65-4 the most beginning 4 will be append to the front at last
    #sent_tensor = np.array.zeros((1,61,5))
    #sent_counter = 0
    doc_list =[]
    for (ind,sents) in enumerate(tree)
         sent_list = []
        for (key,currnet_node) in sents
            #@show key 
            if key == 0;continue;end
            #currnet_node = sents[key]
            word_list = []
            for i in range(1,conv_length)
                #@show currnet_node.word
                if currnet_node.word != "ROOT";push!(word_list,currnet_node.word)
                else; push!(word_list,currnet_node.word);end
                if currnet_node.word != "ROOT"; currnet_node = sents[currnet_node.parentindex];end
            end
             push!(sent_list,word_list)
            #@show length(sent_list)
        end 
        header = []
        dummy = repeat(["ROOT"],conv_length)
        for i in range(1,conv_length-1);push!(header,vcat(dummy[1:conv_length-i-1], sent_list[1][1:i+1]));end
        sent_list = vcat(header,sent_list)
        while length(sent_list) < max_len;push!(sent_list,dummy);end
        currnet_label = labels_dict[ind]
        class_dummy = repeat([currnet_label],conv_length)
        push!(sent_list,class_dummy)
        #@show length(sent_list)
        push!(doc_list,sent_list)
            end     
    @show length(doc_list)
    return doc_list
end

set_conv_sent (generic function with 1 method)

In [10]:
function add_tree2vocab(sent, vocab)
    
    for (j, each_word) in enumerate(sent[1:end-1])
        for (l, each_field) in enumerate(each_word)
            if each_field in keys(vocab);continue
            elseif each_field == 0;continue
            elseif each_field == "ROOT";continue
                else;vocab[each_field] += 1;end
        end
    end
end

add_tree2vocab (generic function with 1 method)

In [11]:
function merge_two(revs, tree)
    counter=1
    for i in revs
        sent2 = tree[counter]
        counter += 1
        i["tree"] = sent2
    end    
    return revs
        
end

merge_two (generic function with 1 method)

In [13]:
#it should read 1200 byte and return 1x300 matrix, but due to there is not read byte method in Julia 
function load_bin_vec(fname, vocab)
pf(s)=return parse(Int,s)
pc(s)=return convert(Char,s[1])
word_vecs = Dict()
    open(fname, "r") do f
                @show header = readline(f)
                vocab_size, layer1_size = map(pf, split(header))
                @show binary_len = sizeof(Float32) * layer1_size
                #@show  binary_len = layer1_size
             for line in collect(1:vocab_size)
                word=[]
            #println("enter to while \t")    
                while true 
                           ch=read(f,1)
                           ch=convert(Char,ch[1])
                            if ch == ' '
                                word = join(word,"")
                                break
                            end
                            if ch != '\n';
                                push!(word,ch);
                            end
                    end
       # println("Exiting from while \t")    
                if word in keys(vocab)
                   word_vecs[word]=reinterpret(Float32,read(f,binary_len))
#                    w = Array{Float64}(map(pc,temp))
#                    vec_norm = norm(w)
#                    word_vecs[word]= w./vec_norm
                else
                read(f,binary_len)
                end

            end    
    end;
return word_vecs
end

load_bin_vec (generic function with 1 method)

In [14]:
@doc reinterpret

```
reinterpret(type, A)
```

Change the type-interpretation of a block of memory. For arrays, this constructs a view of the array with the same binary data as the given array, but with the specified element type. For example, `reinterpret(Float32, UInt32(7))` interprets the 4 bytes corresponding to `UInt32(7)` as a [`Float32`](@ref).

# Examples

```jldoctest
julia> reinterpret(Float32, UInt32(7))
1.0f-44

julia> reinterpret(Float32, UInt32[1 2 3 4 5])
1×5 reinterpret(Float32, ::Array{UInt32,2}):
 1.4013e-45  2.8026e-45  4.2039e-45  5.60519e-45  7.00649e-45
```


In [15]:
function add_unknown_words(word_vecs, vocab, min_df=1, k=300)
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for (word,w) in vocab
        if word ∉ keys(word_vecs) && vocab[word] >= min_df
            @show word  
            word_vecs[word] = (rand(k).*0.5).- 0.25
        end
    end
end

add_unknown_words (generic function with 3 methods)

In [16]:
function get_W(word_vecs, k=300)
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    @show vocab_size = length(word_vecs)
    word_idx_map = Dict()
    W = zeros((vocab_size+1, k))            
    W[1,:] =  zeros(300)
    i = 1
    for (word,w) in word_vecs
        W[i,:] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    end
    return W, word_idx_map
    end

get_W (generic function with 2 methods)

In [19]:
w2v_file = "google_w2v.bin"   
sent_file = "Data/TREC_all.txt"
#tree_file = "Data/data.jld2" # hdf5 wrtoe 
label_file = "Data/label_all.txt"
label_dict = get_labels(label_file);
split_dict = get_split(5952) ;
       
revs, vocab = build_data_cv(sent_file, split_dict, label_dict);
function dfun(d::Dict);return d["num_words"];end
max_l,maxIndex = findmax(map(dfun, revs))
    
all_tree = load("Data/data.jld2","data"); # use jld2 method
data_sibling = set_sibling2(all_tree,label_dict,max_l+8);
data_tree = set_conv_sent(all_tree,label_dict,max_l+8); 
#summary.(data_tree)

new_data_tree = []
for (ind,l) in enumerate(data_tree)
    new_list=[]
    for (ind2,l2) in enumerate(l);push!(new_list,vcat(data_tree[ind][ind2],data_sibling[ind][ind2]));end
    push!(new_data_tree,new_list)
end
data_tree = new_data_tree
#@show length.(new_data_tree)
for i in data_tree;add_tree2vocab(i, vocab);end
@show length(vocab)
revs = merge_two(revs,data_tree);

sent_num = length(tree) = 5952


┌ Warning: `range(start, stop)` (with neither `length` nor `step` given) is deprecated, use `range(start, stop=stop)` instead.
│   caller = set_conv_sent(::Array{Any,1}, ::Dict{Any,Any}, ::Int64) at In[9]:15
└ @ Main ./In[9]:15
┌ Warning: `range(start, stop)` (with neither `length` nor `step` given) is deprecated, use `range(start, stop=stop)` instead.
│   caller = set_conv_sent(::Array{Any,1}, ::Dict{Any,Any}, ::Int64) at In[9]:26
└ @ Main ./In[9]:26


length(doc_list) = 5952
length(vocab) = 10096


In [20]:
println("data loaded!")
println("number of sentences: ", length(revs))
println("vocab size: " ,length(vocab))
println("max sentence length: " ,max_l)
println("loading word2vec vectors...")
w2v = load_bin_vec(w2v_file, vocab)
#w2v=Dict()
println("word2vec loaded!")
println("num words already in word2vec: ",length(w2v))
vocab["ROOT"]=1
vocab["*START*"]=1
vocab["*STOP*"]=1
vocab["*ZERO*"]=1   
add_unknown_words(w2v, vocab)
println("num words already in word2vec: ",length(w2v))
W, word_idx_map = get_W(w2v)
rand_vecs = Dict()
add_unknown_words(rand_vecs, vocab)
W2, _ = get_W(rand_vecs)
save("TREC_sib.jld2","datas",[revs, W, W2, word_idx_map, vocab])  
println("dataset created!")

data loaded!
number of sentences: 5952
vocab size: 10096
max sentence length: 37
loading word2vec vectors...
header = readline(f) = "3000000 300"
binary_len = sizeof(Float32) * layer1_size = 1200
word2vec loaded!
num words already in word2vec: 9040
word = "1881"
word = "Vlaja"
word = "slane"
word = "24-disc"
word = "1880"
word = "Pesth"
word = "1863"
word = "Syrian_held"
word = "39"
word = "E=mc2"
word = "nuclear-powered"
word = "17"
word = "``"
word = "Hohenzollerns"
word = "14"
word = "a"
word = "197"
word = "Catch-22"
word = "401"
word = "Rednitz"
word = "1973"
word = "21"
word = "X_Man"
word = "2001"
word = "Gumee"
word = "Philebus-like"
word = "Russian_led"
word = "1939"
word = "co-starred"
word = "1923"
word = "1997"
word = "1942"
word = "1969"
word = "61"
word = "H.G."
word = "nuclear_powered"
word = "three-letter"
word = "second_most_used"
word = "49"
word = "D_Day"
word = "455-yard"
word = "1847"
word = "policy_making"
word = "Bergen-Jacqueline"
word = "193"
word = "Philebus_l

word = "1945"
word = "#1"
word = "16-year"
word = "five_zone"
word = "napsylate"
word = "mehitabel"
word = "\$836"
word = "cwt."
word = "Depreciates"
word = "second_biggest_selling"
word = "beer-producing"
word = "writer_journalist"
word = "SysRq"
word = "1981"
word = "35824"
word = "22"
word = "second-highest"
word = "1967"
word = "pig_the"
word = "second_lightest"
word = "1815"
word = "swift."
word = "half_Irish"
word = "387"
word = "six_foot"
word = "M&Ms"
word = "heliologist"
word = "est-ce"
word = "192"
word = "15_minute"
word = "object_oriented"
word = "14_mile"
word = "187s"
word = "brothers-in-law"
word = "bicornate"
word = "239"
word = "737"
word = "1865"
word = "8/28/1941"
word = "hydrosulfite"
word = "A_Tisket"
word = "56_game"
word = "pyramid_builders"
word = "McPugg"
word = "non_mechanical"
word = "oil-producing"
word = "anti-AIDS"
word = "''"
word = "dumb_but_loveable"
word = "100"
word = "1953"
word = "48"
word = "three_ton"
word = "anti_AIDS"
word = "up-to-date"
word = 

vocab_size = length(word_vecs) = 10097
word = "ferret"
word = "tiles"
word = "Apple"
word = "NAFTA"
word = "shouldn"
word = "chocolate"
word = "fight"
word = "Comaneci"
word = "1881"
word = "qigong"
word = "Empire"
word = "Thalia"
word = "Havlicek"
word = "Casablanca"
word = "whose"
word = "manufacture"
word = "photosynthesis"
word = "incoherent"
word = "Vlaja"
word = "creams"
word = "Mormons"
word = "slane"
word = "borders"
word = "Ottawa"
word = "Kosovo"
word = "star"
word = "Wimbledon"
word = "24-disc"
word = "NY"
word = "Puppy"
word = "Notch"
word = "plan"
word = "rises"
word = "1880"
word = "those"
word = "Arch"
word = "Pesth"
word = "reform"
word = "Libya"
word = "sugar"
word = "months"
word = "destination"
word = "Coral"
word = "copper"
word = "general"
word = "1863"
word = "Jayne"
word = "toys"
word = "Felicity"
word = "underground"
word = "OJ"
word = "actually"
word = "Narragansett"
word = "dye"
word = "canal"
word = "these"
word = "backup"
word = "least"
word = "caught"
word 

word = "Gold"
word = "six-footed"
word = "published"
word = "Minneapolis"
word = "Needle"
word = "1952"
word = "stat"
word = "one-pound"
word = "maintain"
word = "beethoven"
word = "20th"
word = "celtic"
word = "currency"
word = "computers"
word = "Buffalo"
word = "e_commerce"
word = "look"
word = "dozen"
word = "mob"
word = "annual"
word = "Badaling"
word = "extraordinary"
word = "enough"
word = "photographer"
word = "Colonel"
word = "longest-lived"
word = "Edition"
word = "Arnold"
word = "cash-conscious"
word = "laser"
word = "Packers"
word = "Daisy"
word = "complete"
word = "into"
word = "15-year-old"
word = "isolationist"
word = "surrendered"
word = "animals"
word = "pop"
word = "Wiggins"
word = "graces"
word = "intestines"
word = "occupation"
word = "Berlin"
word = "sculpture"
word = "empty"
word = "archery"
word = "successfully"
word = "doing"
word = "City"
word = "ESPN"
word = "lotion"
word = "Superman"
word = "Everybody"
word = "force"
word = "boss"
word = "round"
word = "accom

word = "exercises"
word = "Diplomacy"
word = "84"
word = "innings"
word = "mined"
word = "Marine"
word = "Wharton"
word = "Massachusetts"
word = "starting"
word = "Leon"
word = "steamboat"
word = "funded"
word = "cold-water"
word = "mid"
word = "hydrogen"
word = "boxcars"
word = "questions"
word = "saltpeter"
word = "rice"
word = "blank"
word = "Countries"
word = "Kurt"
word = "girl"
word = "human"
word = "tunnel"
word = "whole"
word = "Dead"
word = "1983"
word = "apartments"
word = "Binney"
word = "volleyball"
word = "motorcycles"
word = "item"
word = "underworld"
word = "Barkis"
word = "itself"
word = "rarely"
word = "Grape"
word = "HUGO"
word = "fishmarket"
word = "Trojan"
word = "lived"
word = "big"
word = "Andrews"
word = "Mother"
word = "devil"
word = "chartered"
word = "quit"
word = "edge"
word = "experts"
word = "eight-ball"
word = "Breony"
word = "College"
word = "Prelude"
word = "average"
word = "newspaper"
word = "Abner"
word = "wrote"
word = "Brethren"
word = "wrap"
word = 

...

word = "once-a-year"
word = "hourglass"
word = "attacked"
word = "proud"
word = "completed"
word = "e-coli"
word = "Wild"
word = "witch"
word = "gave"
word = "half-life"
word = "Allan"
word = "bleach"
word = "eggs"
word = "blue"
word = "synonymous"
word = "Mexican"
word = "Barrymore"
word = "water"
word = "non-contagious"
word = "1893"
word = "Moog"
word = "survival"
word = "following"
word = "shots"
word = "remake"
word = "veronica"
word = "man-made"
word = "Alabama"
word = "turn"
word = "folklore"
word = "cranes"
word = "Sondheim"
word = "fit"
word = "28"
word = "Zambia"
word = "deposit"
word = "threat"
word = "southern"
word = "sense"
word = "Randy"
word = "'Hara"
word = "else"
word = "sensitive"
word = "Mack"
word = "valve"
word = "cats"
word = "Lisa"
word = "compete"
word = "hypnotherapy"
word = "turning"
word = "superb"
word = "dial"
word = "Stein"
word = "ecological"
word = "non_alcoholic"
word = "revolution"
word = "degrees"
word = "dragged"
word = "back"
word = "25"
word = "ma

word = "palindromic"
word = "exercise"
word = "mummified"
word = "heir"
word = "\$"
word = "E_mail"
word = "propylene"
word = "Energy"
word = "compatible"
word = "without"
word = "Brazil"
word = "America"
word = "least-populated"
word = "chronicles"
word = "hike"
word = "Taylor"
word = "Marx"
word = "guidance"
word = "Kippur"
word = "Grenada"
word = "Dungri"
word = "Louse"
word = "intake"
word = "seat"
word = "Everything"
word = "difference"
word = "scientific"
word = "lion"
word = "niece"
word = "four-volume"
word = "Horlick"
word = "1960s"
word = "Ozzie"
word = "pulse"
word = "Maximo"
word = "bikini"
word = "Mansfield"
word = "Homer"
word = "Rom"
word = "somebody"
word = "chance"
word = "believe"
word = "Montmartre"
word = "propaganda"
word = "drills"
word = "."
word = "mean"
word = "Cassidey"
word = "Neil"
word = "split"
word = "boosters"
word = "East-West"
word = "England"
word = "\$85"
word = "guitarist"
word = "swallow"
word = "Olive"
word = "Fiedler"
word = "Clock"
word = "use"


word = "i"
word = "enzymes"
word = "Morgan"
word = "Turin"
word = "donating"
word = "name.."
word = "NNP"
word = "Faso"
word = "Luke"
word = "roommates"
word = "Wee"
word = "ants"
word = "Bendix"
word = "BMW"
word = "tumbled"
word = "echidna"
word = "ever"
word = "marble"
word = "bloodhound"
word = "develop"
word = "baseemen"
word = "satelite"
word = "Duane"
word = "liters"
word = "gets"
word = "limited"
word = "elevation"
word = "equipment"
word = "Lourve"
word = "Muslims"
word = "1951"
word = "mascot"
word = "Bitmap"
word = "motto"
word = "University"
word = "advertises"
word = "equinox"
word = "shipment"
word = "JDR3"
word = "hematoma"
word = "Nevil"
word = "machine"
word = "blind"
word = "Small"
word = "founding"
word = "further"
word = "Kythnos"
word = "Bucher"
word = "Cid"
word = "won-lost"
word = "orca"
word = "kitchen"
word = "tampon"
word = "longer"
word = "Chiefs"
word = "snore"
word = "carried"
word = "pushed"
word = "x_rays"
word = "Francisco"
word = "LP"
word = "Sisters"
w

word = "classification"
word = "wielded"
word = "Poles"
word = "lightning"
word = "Parker"
word = "Do"
word = "movie"
word = "inducted"
word = "Square"
word = "equilibrium"
word = "attire"
word = "enlivens"
word = "tender"
word = "strikes"
word = "Flying"
word = "19th-century"
word = "epicenter"
word = "Heaven"
word = "developmental"
word = "hamburgers"
word = "couple"
word = "within"
word = "profits"
word = "158"
word = "themes"
word = "full-length"
word = "Pooh"
word = "chain"
word = "nominee"
word = "Century_Fox"
word = "pronounced"
word = "Hernando"
word = "quarters"
word = "distinguishing"
word = "seats"
word = "packages"
word = "tanker"
word = "canker"
word = "switch"
word = "16_year"
word = "sired"
word = "Ad"
word = "bubblegum"
word = "Hirsch"
word = "Alyssa"
word = "consist"
word = "shadow"
word = "crystals"
word = "Menace"
word = "rugs"
word = "bathing"
word = "USSR"
word = "peninsula"
word = "Thanksgiving"
word = "pizza"
word = "bomb"
word = "Thor"
word = "Cop"
word = "build

word = "Self_Portrait"
word = "goalie"
word = "oddsmaker"
word = "Peugeot"
word = "pseudonym"
word = "Wal_Mart"
word = "ingredient"
word = "quantity"
word = "explosion"
word = "yet"
word = "fences"
word = "consortium"
word = "Iron"
word = "ignores"
word = "meat"
word = "_"
word = "larger"
word = "triplets"
word = "disorder"
word = "Dow"
word = "Enterprise"
word = "fourth"
word = "198"
word = "deltiologist"
word = "magnetar"
word = "sales"
word = "adding"
word = "plugged"
word = "Playstation"
word = "place"
word = "Burnford"
word = "From"
word = "poetic"
word = "small"
word = "cannon"
word = "bound"
word = "bears"
word = "expect"
word = "Bella"
word = "Ocean"
word = "attract"
word = "Miss"
word = "plural"
word = "Answers.com"
word = "Six"
word = "Lilly"
word = "Emperor"
word = "refusing"
word = "eruption"
word = "Of"
word = "maybe"
word = "vermouth"
word = "Rummy"
word = "dartboard"
word = "carelessness"
word = "rulebook"
word = "Sicily"
word = "732"
word = "Spritzer"
word = "joint"
wor